# TechJob - 社群網路分析
**Group4 - Study4** <br>
資料來源: PTT TechJob(科技工作)版<br>
資料筆數: 1647筆

In [34]:
#! pip install pyvis networkx pandas numpy

In [35]:
import pandas as pd
import numpy as np
import pyvis
import networkx as nx
import IPython

In [36]:
raw_data = pd.read_csv('./data/Tech_Job_OriginalData.csv')
raw_data = raw_data[raw_data.artComment != '[]'] # 刪除沒有comment的文章
raw_data = raw_data.sample(round(raw_data.shape[0]/2)) # 只取資料集的1/10，以利後續畫圖
print(raw_data.shape)
raw_data.head()


(818, 11)


,system_id,artUrl,artTitle,artDate,artPoster,artCatagory,artContent,artComment,e_ip,insertedDate,dataSource
1348,1349,https://www.ptt.cc/bbs/Tech_Job/M.1648989020.A...,[新聞]月薪沒4萬不做被罵眼高手低 名校理工生,2022-04-03 20:30:17,zzzz8931,Tech_Job,月薪沒4萬不做被罵眼高手低 名校理工生不服：這社會奴啊\nhttps://www.setn....,"[{""cmtStatus"": ""→"", ""cmtPoster"": ""IBIZA"", ""cmt...",NaN,2022-04-04 02:15:14,ptt
243,244,https://www.ptt.cc/bbs/Tech_Job/M.1537842083.A...,[請益]廣達產品工程師？,2018-09-25 02:15:21,jp956956,Tech_Job,大家好\n最近在104上收到廣達的通知\n職缺是 產品工程師\n\n接到人資打電話來要我決定...,"[{""cmtStatus"": ""推"", ""cmtPoster"": ""aaabbb111"", ...",110.50.167.238,2018-10-08 01:02:52,ptt
1334,1335,https://www.ptt.cc/bbs/Tech_Job/M.1647956730.A...,[問卷]受僱者/求職者經驗訪談(車馬費500),2022-03-22 21:45:27,karaslavi,Tech_Job,*經版主同意貼文*\n\n大家好，本人正在研發一款關於「受僱者與求職者相關之app」\n目前...,"[{""cmtStatus"": ""推"", ""cmtPoster"": ""gaea0127"", ""...",150.116.211.109,2022-03-23 01:14:08,ptt
1296,1297,https://www.ptt.cc/bbs/Tech_Job/M.1644281277.A...,公益:履歷健檢&面試建議,2022-02-08 08:47:55,olaola,Tech_Job,感謝PTT科技版在求職過程中指點迷津。\n新年新希望，希望自己在有點經驗的時候，在履歷修改及...,"[{""cmtStatus"": ""推"", ""cmtPoster"": ""asz142536"", ...",114.136.1.180,2022-02-09 01:36:09,ptt
1396,1397,https://www.ptt.cc/bbs/Tech_Job/M.1652748579.A...,Re:[請益]該辭職嗎？,2022-05-17 08:49:37,hidog,Tech_Job,"到哪都會遇到垃圾系統\n還沒聽過哪家公司沒有垃圾code, legacy code, 歷史包...","[{""cmtStatus"": ""推"", ""cmtPoster"": ""KaoZen"", ""cm...",36.227.209.27,2022-05-18 02:40:20,ptt


In [37]:
# 取出 commenter
def getComtInfo(com):
    cmters, cmt_statuss = [],[]
    com = eval(com)
    for i in com:
        # print(i)
        cmters.append(i['cmtPoster'])
        cmt_statuss.append(i['cmtStatus'])
        return pd.Series([cmters, cmt_statuss])


raw_data[['artComter','artStatus']] = raw_data['artComment'].apply(lambda r: getComtInfo(r))

In [38]:
raw_data = raw_data.explode(['artComter', 'artStatus'])

socail_data = raw_data[['artUrl', 'artPoster', 'artComter', 'artStatus']]
socail_data.head(10)


,artUrl,artPoster,artComter,artStatus
1348,https://www.ptt.cc/bbs/Tech_Job/M.1648989020.A...,zzzz8931,IBIZA,→
243,https://www.ptt.cc/bbs/Tech_Job/M.1537842083.A...,jp956956,aaabbb111,推
1334,https://www.ptt.cc/bbs/Tech_Job/M.1647956730.A...,karaslavi,gaea0127,推
1296,https://www.ptt.cc/bbs/Tech_Job/M.1644281277.A...,olaola,asz142536,推
1396,https://www.ptt.cc/bbs/Tech_Job/M.1652748579.A...,hidog,KaoZen,推
485,https://www.ptt.cc/bbs/Tech_Job/M.1555418927.A...,moonth66,moonth66,→
852,https://www.ptt.cc/bbs/Tech_Job/M.1594338574.A...,YenFuOne,pttccbbs,→
1206,https://www.ptt.cc/bbs/Tech_Job/M.1634019273.A...,mile022,goodga,推
961,https://www.ptt.cc/bbs/Tech_Job/M.1609296515.A...,njpp,wulouise,推
977,https://www.ptt.cc/bbs/Tech_Job/M.1611797588.A...,howardfang,RTA,推


In [39]:
socail_data.artStatus.unique()

array(['→', '推', '噓'], dtype=object)

### 簡單劃出文章、發文者、評論者的網路

In [40]:
# 建立邊的資料
import random

po_df = socail_data[['artPoster','artUrl']].drop_duplicates().rename(columns = {'artPoster':'src','artUrl':'dis'})
sample_url = random.choices(po_df.dis.unique().tolist(),k=300)
po_df = po_df[po_df.dis.isin(sample_url)]
re_df = socail_data[['artComter','artUrl','artStatus']].rename(columns = {'artComter':'src','artUrl':'dis','artStatus':'weight'})
re_df = re_df[re_df.dis.isin(sample_url)]
re_df = re_df[~re_df['src'].isna()]



def convertStatus(s):
  if s == '推':
    return 2
  elif s == '→': # '→' 也算是推
    return 1
  else :
    return -1
re_df['weight'] = re_df['weight'].map(convertStatus)
re_df = re_df.groupby(['src','dis']).sum().reset_index()

# 設定分數低的為紅色
def getColor(w):
  if w>0:
    return 'green'
  else:
    return 'red'
re_df['color'] = re_df.weight.map(getColor)



In [41]:
re_df

,src,dis,weight,color
0,AGODC,https://www.ptt.cc/bbs/Tech_Job/M.1667705715.A...,2,green
1,Agclean,https://www.ptt.cc/bbs/Tech_Job/M.1599020209.A...,2,green
2,Ayanami5566,https://www.ptt.cc/bbs/Tech_Job/M.1615349659.A...,2,green
3,BlueBird5566,https://www.ptt.cc/bbs/Tech_Job/M.1594009552.A...,1,green
4,BlueBird5566,https://www.ptt.cc/bbs/Tech_Job/M.1665070387.A...,-1,red
...,...,...,...,...
246,zinn,https://www.ptt.cc/bbs/Tech_Job/M.1624071419.A...,2,green
247,zombiechen,https://www.ptt.cc/bbs/Tech_Job/M.1671691925.A...,-1,red
248,zxcv1478,https://www.ptt.cc/bbs/Tech_Job/M.1636471042.A...,2,green
249,zxp9505007,https://www.ptt.cc/bbs/Tech_Job/M.1668618080.A...,2,green


In [42]:
# 編碼設定  
import _locale
_locale._getdefaultlocale = (lambda *args: ['en_US', 'utf8'])
_locale._getdefaultlocale

<function __main__.<lambda>(*args)>

In [43]:
# 人為綠色節點
# po文為橘色節點

netWork = pyvis.network.Network(notebook=True, cdn_resources='in_line',directed=True)
person = list(set(po_df.src.unique().tolist()+re_df.src.unique().tolist()))
url = po_df.dis.unique().tolist()
netWork.add_nodes(
    nodes = person,
    value = [1 for i in range(len(person))],
    color = ['#66CDAA' for i in range(len(person))],
    title = person
)
netWork.add_nodes(
    nodes = url,
    value = [2 for i in range(len(url))],
    color = ['#FFB366' for i in range(len(url))],
    title = url
)
for i in po_df.to_numpy():
  netWork.add_edge(i[0],i[1],width = 2,color='grey')
for i in re_df.to_numpy():
  netWork.add_edge(i[0],i[1],width = 2,color=i[3])

# 設定layout
netWork.repulsion()

netWork.show("./week14/basic_netWork.html")
# IPython.display.HTML('./week14/basic_netWork.html')

./week14/basic_netWork.html


以上可以看出多數的文章留言偏少，不過大多都是'推'

In [44]:
# 人為綠色節點
# po文為橘色節點

netWork = pyvis.network.Network(notebook=True, cdn_resources='in_line',directed=True)
person = list(set(po_df.src.unique().tolist()+re_df.src.unique().tolist()))
url = po_df.dis.unique().tolist()
netWork.add_nodes(
    nodes = person,
    value = [1 for i in range(len(person))],
    color = ['#66CDAA' for i in range(len(person))],
    title = person
)
netWork.add_nodes(
    nodes = url,
    value = [2 for i in range(len(url))],
    color = ['#FFB366' for i in range(len(url))],
    title = url
)
for i in po_df.to_numpy():
  netWork.add_edge(i[0],i[1],width = 2,color='grey')
for i in re_df.to_numpy():
  netWork.add_edge(i[0],i[1],width = 2,color=i[3])

# 設定layout
netWork.repulsion()

netWork.show("./week14/basic_netWork.html")
# IPython.display.HTML('./week14/basic_netWork.html')

./week14/basic_netWork.html


### 計算網友之間的關係

In [45]:
pos_cmt = socail_data.copy()
pos_cmt = pos_cmt[~pos_cmt.artComter.isna()]
pos_cmt.head(10)

,artUrl,artPoster,artComter,artStatus
1348,https://www.ptt.cc/bbs/Tech_Job/M.1648989020.A...,zzzz8931,IBIZA,→
243,https://www.ptt.cc/bbs/Tech_Job/M.1537842083.A...,jp956956,aaabbb111,推
1334,https://www.ptt.cc/bbs/Tech_Job/M.1647956730.A...,karaslavi,gaea0127,推
1296,https://www.ptt.cc/bbs/Tech_Job/M.1644281277.A...,olaola,asz142536,推
1396,https://www.ptt.cc/bbs/Tech_Job/M.1652748579.A...,hidog,KaoZen,推
485,https://www.ptt.cc/bbs/Tech_Job/M.1555418927.A...,moonth66,moonth66,→
852,https://www.ptt.cc/bbs/Tech_Job/M.1594338574.A...,YenFuOne,pttccbbs,→
1206,https://www.ptt.cc/bbs/Tech_Job/M.1634019273.A...,mile022,goodga,推
961,https://www.ptt.cc/bbs/Tech_Job/M.1609296515.A...,njpp,wulouise,推
977,https://www.ptt.cc/bbs/Tech_Job/M.1611797588.A...,howardfang,RTA,推


該網友針對該篇文章有多少次評論 <br>
if 正面回覆則 +1

In [46]:
pos_cmt['score'] = pos_cmt['artStatus'].apply(lambda r: convertStatus(r))
pos_cmt = pos_cmt.groupby(['artComter','artPoster']).sum('score').reset_index()
pos_cmt = pos_cmt[pos_cmt.score>=0]
pos_cmt = pos_cmt[pos_cmt.artComter != pos_cmt.artPoster]
# pos_cmt.score = 1
pos_cmt

,artComter,artPoster,score
0,AGODC,sxy67230,2
1,ASEglobal,shueisha26,2
2,AboveTheRim,kaube,2
3,Agclean,catcatcatcat,2
4,AgentSkye56,dnol,2
...,...,...,...
803,zombiechen,ks32437,2
806,zxcasd840510,berna09f,2
807,zxcv1478,KAINTS,2
808,zxp9505007,AAAB,2


計算 matrix

In [47]:
pers = np.unique(pos_cmt[['artComter', 'artPoster']])
mat = pd.pivot_table(pos_cmt,index = 'artComter', columns = 'artPoster' ,values='score' ).fillna(0)\
  .reindex(columns=pers, index=pers, fill_value=0).to_numpy() # 有向圖
print(mat.shape)
tri = (np.tril(mat,-1).T + np.triu(mat,1))
mat_s = tri+tri.T # 無向圖


(994, 994)


In [48]:
def random_color():
  r = lambda: random.randint(0,255)
  return '#%02X%02X%02X' % (r(),r(),r())

# input matrix, output graph
def matPresentGraph(mat:np.array,node_id:list,node_type:list=None,node_value:list = None,directed = True,edge_color=None) -> pyvis.network.Network:

  if directed != True:
    if (mat == mat.transpose()).all():
      print('matrix is Symmetric')
    else:
      print('matrix is not Symmetric')
  if node_type == None:
    c = random_color()
    node_colors = [c for i in range(len(node_id))]
    node_type = [" " for i in range(len(node_id))]
  else:
    node_color_map = {}
    for i in set(node_type):
      while True:
        c = random_color()
        if c not in node_color_map.values():
          break
      node_color_map[i] = c
    node_colors = [node_color_map[i] for i in node_type]
  # print(node_colors)
  if edge_color == None:
    edge_color = random_color()
  edge_color = random_color()
  if node_value == None:
    node_value = [1 for i in range(len(node_id))]

  net = pyvis.network.Network(notebook=True, directed = directed, cdn_resources='in_line')
  
  titles_list = []
  for i ,j in zip(node_id,node_type):
    titles_list.append(str(i)+":"+str(j))

  net.add_nodes(
      nodes = node_id,
      value = node_value,
      label = node_id,
      title = titles_list,
      color = node_colors
  )

  for row in range(len(node_id)):
    for col in range(len(node_id)):
      if mat[row][col]>0.:
        net.add_edge(
            node_id[row],node_id[col],width = mat[row][col],color = edge_color,title = mat[row][col]
        )
  net.repulsion()
  return net


In [49]:
# 定義發文者（po）、評論者(cmt)、兩者兼具(both)
node_type = []

cmt_list = pos_cmt['artComter'].unique().tolist()
po_list = pos_cmt['artPoster'].unique().tolist()
both_list = list(set(cmt_list) & set(po_list))
for p in pers:
  if p in both_list:
    node_type.append('both')

  elif p in cmt_list:
    node_type.append('cmt')

  elif p in po_list:
    node_type.append('po')

In [50]:
net = matPresentGraph(mat = mat,node_id = pers,node_type = node_type)
net.show("./week14/pers_netWork.html")
# IPython.display.HTML('/week14/pers_netWork.html')

./week14/pers_netWork.html


### 計算eigenvector centrality （無向圖）

In [51]:
# networkx.eigenvector_centrality(nx_Graph) 將nx.Graph轉為無向圖
# 每個人都會有各自的eigenvalue，代表每個人的重要程度
eigenvec = np.array(list(nx.eigenvector_centrality(nx.Graph(mat_s),max_iter = 10000).values()))
# 進行normalize，讓eigenvalue不要太零散
eigenvec = (eigenvec-np.min(eigenvec))/(np.max(eigenvec)-np.min(eigenvec))

In [52]:
q = np.quantile(eigenvec,[.2,.4,.6,.8]) # 根據不同的quantile給他不同的value => node size
q

array([0.00000000e+00, 0.00000000e+00, 3.50562894e-21, 9.27422472e-03])

In [53]:
node_value = []


for i in eigenvec:

  if i > q[3]:
    node_value.append(25)
  elif i >q[2]:
    node_value.append(20)
  elif i>q[1]:
    node_value.append(15)
  elif i>q[0]:
    node_value.append(10)
  else:
    node_value.append(5)
net = matPresentGraph(mat = mat,node_id = pers,node_type = node_type,node_value = node_value,directed=False)
net.show("./week14/pers_eig_netWork.html")
# IPython.display.HTML('./week14/pers_eig_netWork.html')

matrix is not Symmetric
./week14/pers_eig_netWork.html


我們可以發現有多次評論的人，或是評論的文章同時有多人評論的，重要程度愈高

### betweeness centrality (無向圖)

In [54]:
bet = np.array(list(nx.betweenness_centrality(nx.Graph(mat_s)).values()))
bet = (bet-np.min(bet))/(np.max(bet)-np.min(bet))
bet_q = np.quantile(bet,[.2,.4,.6,.8])

In [55]:
node_value = []
for i in bet:
  if i > bet_q[3]:
    node_value.append(25)
  elif i > bet_q[2]:
    node_value.append(20)
  elif i>bet_q[1]:
    node_value.append(15)
  elif i>bet_q[0]:
    node_value.append(10)
  else:
    node_value.append(5)

net = matPresentGraph(mat = mat,node_id = pers,node_type = node_type,node_value=node_value)
net.show("./week14/pers_bet_netWork.html")
# IPython.display.HTML('./week14/pers_bet_netWork.html')

./week14/pers_bet_netWork.html


`評論較多人或被較多人評論會有比較大的重要程度`

### 重新整理 graph

Transitivity/Density/Distance/Diameter/Clustering

計算最大的subgraph 的 measure

In [56]:
# 計算max subgraph
G = nx.Graph(mat_s)
G_sub = sorted(nx.connected_components(G), key=len, reverse=True)
G_max_sub = G.subgraph(G_sub[0])
# # 重新定義 mat
sub_mat = nx.adjacency_matrix(G_max_sub).todense()
node_idx = list(G_max_sub.nodes)
sub_pers = pers[node_idx]


畫出最大subgraph

In [57]:
net = matPresentGraph(mat=sub_mat,node_id=sub_pers,directed=False)
net.show("./week14/max_sub.html")
# IPython.display.HTML('./week14/max_sub.html')

matrix is Symmetric
./week14/max_sub.html


計算transitivity

In [58]:
nx.transitivity(nx.Graph(sub_mat))

0.010752688172043012

計算density 及 distance

In [59]:
nx.density(nx.Graph(sub_mat))

0.007194742303701141

In [60]:
nx.average_shortest_path_length(nx.Graph(sub_mat))

5.272408624466736

計算diameter

In [61]:
nx.diameter(nx.Graph(sub_mat))

13

計算clustering

In [62]:
nx.average_clustering(nx.Graph(sub_mat))

0.005843532223550064

# 結果分析

從 basic_network 發現
- 直覺可見最多連線的三個中心節點分別為`qazxc1156892`、`zzzz8931`、`orz44444`

接著從網友之間的關係得出 (pers_Network) :
- `qazxc1156892`既是poster也是commenter
- `zzzz8931`及`orz44444`為poster
<br>

又在圖中發現，至少有10個input edge的節點除了上述三個還有 :<br>
`jeff0025`、`dakkk`、`keel90135`、`dolinian`、`waitrop`

從max_subgraph看出
- edge最大為6，其為節點 `qazxc1156892` 和 `jeff0025` 所連接，又與 `zzzz8931` 形成一個較大的三角關係
- edge第二大為4，為節點 `orz44444` 與 `dakkk` 連結成
<br><br>
而`jeff0025` 與 `dakkk` 既是poster也是commenter

In [63]:
max_edge_node = raw_data[raw_data['artPoster'].isin(['qazxc1156892', 'jeff0025', 'zzzz8931'])]
max_edge_node.head()

,system_id,artUrl,artTitle,artDate,artPoster,artCatagory,artContent,artComment,e_ip,insertedDate,dataSource,artComter,artStatus
1348,1349,https://www.ptt.cc/bbs/Tech_Job/M.1648989020.A...,[新聞]月薪沒4萬不做被罵眼高手低 名校理工生,2022-04-03 20:30:17,zzzz8931,Tech_Job,月薪沒4萬不做被罵眼高手低 名校理工生不服：這社會奴啊\nhttps://www.setn....,"[{""cmtStatus"": ""→"", ""cmtPoster"": ""IBIZA"", ""cmt...",NaN,2022-04-04 02:15:14,ptt,IBIZA,→
1432,1433,https://www.ptt.cc/bbs/Tech_Job/M.1656566491.A...,[新聞]馬斯克發通牒「不進辦公室就滾蛋」台灣,2022-06-30 13:21:29,zzzz8931,Tech_Job,馬斯克發通牒「不進辦公室就滾蛋」 台灣調查：上班族僅5%青睞WFH\nhttps://www...,"[{""cmtStatus"": ""推"", ""cmtPoster"": ""qazxc1156892...",114.37.137.92,2022-07-01 02:10:42,ptt,qazxc1156892,推
1683,1684,https://www.ptt.cc/bbs/Tech_Job/M.1672102664.A...,[新聞]凜冬將至！ 科技大咖掀裁員潮,2022-12-27 08:57:40,jeff0025,Tech_Job,凜冬將至！ 科技大咖掀裁員潮\n\n\n「如果手上有現金，2023年是最好的投資時機，因為股...,"[{""cmtStatus"": ""推"", ""cmtPoster"": ""moonth66"", ""...",61.218.53.138,2022-12-28 02:08:06,ptt,moonth66,推
1401,1402,https://www.ptt.cc/bbs/Tech_Job/M.1653380613.A...,[新聞]醫師、工程師平均薪資出爐「畢業生致詞」,2022-05-24 16:23:29,qazxc1156892,Tech_Job,新聞標題: 醫師、工程師平均薪資出爐「畢業生致詞」再掀搶才效應\n\n2022-05-24 ...,"[{""cmtStatus"": ""噓"", ""cmtPoster"": ""Tigerman001""...",223.140.218.161,2022-05-25 02:16:18,ptt,Tigerman001,噓
1518,1519,https://www.ptt.cc/bbs/Tech_Job/M.1665374405.A...,[新聞]陽明交大校園就博會12日登場逾80家廠商,2022-10-10 12:00:01,qazxc1156892,Tech_Job,新聞標題: 陽明交大校園就博會12日登場 逾80家廠商釋5000職缺\nhttps://i....,"[{""cmtStatus"": ""噓"", ""cmtPoster"": ""ijk1"", ""cmtC...",119.14.52.53,2022-10-11 02:03:54,ptt,ijk1,噓


In [64]:
second_edge_node = raw_data[raw_data['artPoster'].isin(['orz44444', 'dakkk'])]
second_edge_node.head()

,system_id,artUrl,artTitle,artDate,artPoster,artCatagory,artContent,artComment,e_ip,insertedDate,dataSource,artComter,artStatus
423,424,https://www.ptt.cc/bbs/Tech_Job/M.1551209967.A...,[新聞]「電子零組件業」好缺人才！職缺成長2成3,2019-02-26 19:33:22,orz44444,Tech_Job,https://udn.com/news/story/7269/3666701\n2019-...,"[{""cmtStatus"": ""→"", ""cmtPoster"": ""j320015566"",...",36.233.67.13,2019-02-27 12:32:12,ptt,j320015566,→
832,833,https://www.ptt.cc/bbs/Tech_Job/M.1592850386.A...,[新聞]企業先雇後訓最高補助10.8萬,2020-06-23 02:26:20,orz44444,Tech_Job,https://udn.com/news/story/7269/4653771\n2020-...,"[{""cmtStatus"": ""推"", ""cmtPoster"": ""wdszfefx"", ""...",114.26.224.97,2020-06-24 00:43:43,ptt,wdszfefx,推
489,490,https://www.ptt.cc/bbs/Tech_Job/M.1555768499.A...,[新聞]發大財！中科大廠紛擴廠徵才1300人薪上看7萬,2019-04-20 13:48:48,orz44444,Tech_Job,https://udn.com/news/story/7240/3766934\n2019-...,"[{""cmtStatus"": ""→"", ""cmtPoster"": ""luche"", ""cmt...",1.165.108.247,2019-04-24 12:34:27,ptt,luche,→
429,430,https://www.ptt.cc/bbs/Tech_Job/M.1551759500.A...,[新聞]群創將招募500名工程師博士起薪6.1萬元,2019-03-05 04:12:15,orz44444,Tech_Job,https://udn.com/news/story/7240/3677945\n2019-...,"[{""cmtStatus"": ""推"", ""cmtPoster"": ""dakkk"", ""cmt...",36.235.112.143,2019-03-06 12:54:43,ptt,dakkk,推
885,886,https://www.ptt.cc/bbs/Tech_Job/M.1597405957.A...,[新聞]「5G應用新領域」論壇產官學精英與談為人才找方向,2020-08-14 19:52:30,orz44444,Tech_Job,https://udn.com/news/story/7240/4781960\n2020-...,"[{""cmtStatus"": ""噓"", ""cmtPoster"": ""wellkom"", ""c...",36.233.95.105,2020-08-15 00:51:43,ptt,wellkom,噓


從上列分析看出 edge 前兩大相關的 artPoster 都是發布`新聞相關`的文章，也因此在版中被最多人評論 (per_bet_netWork)


而另外發現，`waitrop`的文章title都有 `RE:...`，表示該用戶是在PPT上有較活躍的發文紀錄，且文章都有較高的討論度

In [65]:
edge_node = raw_data[raw_data['artPoster'].isin(['waitrop'])]
edge_node

,system_id,artUrl,artTitle,artDate,artPoster,artCatagory,artContent,artComment,e_ip,insertedDate,dataSource,artComter,artStatus
1676,1677,https://www.ptt.cc/bbs/Tech_Job/M.1671766268.A...,Re:[討論]在科技業一輩子能賺十億嗎?,2022-12-23 11:31:06,waitrop,Tech_Job,"現在怎麼鼓吹炒房的文章在每個版到處發文,\n開始急了?\n其實用數學算一下就知道,\n炒房不...","[{""cmtStatus"": ""推"", ""cmtPoster"": ""verilogTire""...",24.5.140.41,2022-12-25 02:05:57,ptt,verilogTire,推
729,730,https://www.ptt.cc/bbs/Tech_Job/M.1583559781.A...,Re:[討論]工程師似乎還算穩定職業,2020-03-07 13:42:59,waitrop,Tech_Job,"我很像太晚加入這場戰爭了，\n但是還是來分享一下魯蛇工程師的看法,\n工程師絕對不是穩定職業...","[{""cmtStatus"": ""→"", ""cmtPoster"": ""twflash"", ""c...",24.4.131.164,2020-03-08 02:24:46,ptt,twflash,→
1024,1025,https://www.ptt.cc/bbs/Tech_Job/M.1617173153.A...,"Re:[討論]外商在台灣的薪資越來越""神奇""",2021-03-31 14:45:51,waitrop,Tech_Job,在這邊提供一下相反的意見，\n其實，我之前就說過，\n硬體公司薪資最高的就在台灣的台商，\n...,"[{""cmtStatus"": ""推"", ""cmtPoster"": ""softseaweed""...",24.4.131.164,2021-04-01 01:39:48,ptt,softseaweed,推
1659,1660,https://www.ptt.cc/bbs/Tech_Job/M.1670819164.A...,Re:[新聞]台積電最先進製程會留台灣因為薪水低,2022-12-12 12:26:02,waitrop,Tech_Job,"很多訊息都是錯誤或是故意誤導,\n剛好我曾經在Chandler AZ 工作一陣子,\n就在台...","[{""cmtStatus"": ""推"", ""cmtPoster"": ""marsonele"", ...",24.5.140.41,2022-12-13 02:06:37,ptt,marsonele,推
1095,1096,https://www.ptt.cc/bbs/Tech_Job/M.1623622171.A...,Re:[請益]IC設計真的比較好嗎,2021-06-14 06:09:29,waitrop,Tech_Job,爽不爽這種事情很難說，\n即使同公司甚至同個組，\n光是做的項目不同就有天堂跟地獄的差別，\...,"[{""cmtStatus"": ""推"", ""cmtPoster"": ""expiate"", ""c...",24.4.131.164,2021-06-15 02:15:03,ptt,expiate,推
1556,1557,https://www.ptt.cc/bbs/Tech_Job/M.1667463859.A...,Re:[新聞]安謀告高通燒到手機晶片廠,2022-11-03 16:24:17,waitrop,Tech_Job,"哎！\n看到一堆前公司的名字,\n身為一輩子在這些公司跟手機CPU業界打轉的社畜,\n還是有...","[{""cmtStatus"": ""→"", ""cmtPoster"": ""baiya"", ""cmt...",76.132.71.160,2022-11-04 02:06:37,ptt,baiya,→
528,529,https://www.ptt.cc/bbs/Tech_Job/M.1559628584.A...,[面試]Microsoft面試心得,2019-06-04 06:03:42,waitrop,Tech_Job,"很久沒有分享面試心得，\n因為很久沒有換工作了，\n但是2018, 2019年是個非常特殊的...","[{""cmtStatus"": ""推"", ""cmtPoster"": ""southpeace"",...",73.222.35.14,2019-06-06 13:02:32,ptt,southpeace,推


`dolinian`則都是`徵才`相關的文章，且都有固定發文格式，因此推測該帳號是用來協助企業發布徵才相關資訊的

In [66]:
edge_node2 = raw_data[raw_data['artPoster'].isin(['dolinian'])]
edge_node2.head()

,system_id,artUrl,artTitle,artDate,artPoster,artCatagory,artContent,artComment,e_ip,insertedDate,dataSource,artComter,artStatus
872,873,https://www.ptt.cc/bbs/Tech_Job/M.1596454040.A...,[徵才]華碩電腦誠徵軟體工程師,2020-08-03 19:27:17,dolinian,Tech_Job,【公司名稱】\n華碩電腦\n\n【公司地址】\n台北市北投區立德路15號\n\n【職缺】\n...,"[{""cmtStatus"": ""推"", ""cmtPoster"": ""PauFrank5566...",220.135.205.37,2020-08-04 02:14:30,ptt,PauFrank5566,推
254,255,https://www.ptt.cc/bbs/Tech_Job/M.1538462860.A...,Fw:[徵才]快速面試_WeMo徵才中,2018-10-02 06:41:38,dolinian,Tech_Job,作者: dolinian (杜麗娘) 看板: Soft_Job\n標題: [徵才] 快速面試...,"[{""cmtStatus"": ""→"", ""cmtPoster"": ""holsety"", ""c...",NaN,2018-10-08 01:03:11,ptt,holsety,→
936,937,https://www.ptt.cc/bbs/Tech_Job/M.1605077259.A...,[徵才]植茁科技誠徵夥伴（歡迎新鮮人）,2020-11-11 14:47:35,dolinian,Tech_Job,[徵才公司]\n 植茁科技\n \n [關於植茁科技]\n房屋新創 ap...,"[{""cmtStatus"": ""推"", ""cmtPoster"": ""j835111"", ""c...",220.135.205.37,2020-11-12 01:04:54,ptt,j835111,推
1047,1048,https://www.ptt.cc/bbs/Tech_Job/M.1618827868.A...,[徵才]Grab在台招募前/後端/軟體及測試工程師,2021-04-19 18:24:25,dolinian,Tech_Job,[公司名稱]\nGrab\n\n[公司網址]\nhttps://grab.careers\n...,"[{""cmtStatus"": ""推"", ""cmtPoster"": ""gaea0127"", ""...",220.135.205.37,2021-04-20 02:16:29,ptt,gaea0127,推
276,277,https://www.ptt.cc/bbs/Tech_Job/M.1540742201.A...,Fw:[徵才]港商Wisers徵Java/Node/React工程師,2018-10-28 15:50:40,dolinian,Tech_Job,作者: dolinian (杜麗娘) 看板: Soft_Job\n標題: [徵才] 港商Wi...,"[{""cmtStatus"": ""推"", ""cmtPoster"": ""diabloevagto...",36.225.21.131,2018-10-28 12:34:36,ptt,diabloevagto,推
